<a href="https://colab.research.google.com/github/franmazzaro/7506R-2C2023-GRUPO39/blob/main/7506R_TP2_GRUPO39_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librerias**

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV

# Carga de datasets

In [15]:
df_criticas = pd.read_csv('/content/sample_data/train.csv')
df_criticas_test = pd.read_csv('/content/sample_data/test.csv')


In [ ]:
df_criticas.head(4)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo


# Preprocesamiento

In [17]:
# Vectorización de texto usando CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_criticas['review_es'])
X_test = vectorizer.transform(df_criticas_test['review_es'])

# Codificar las etiquetas de sentimiento
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_criticas['sentimiento'])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=25)

In [20]:
# Función para convertir valores
def convertir(valor):
    return "negativo" if valor == 0 else "positivo"

# Bayes Naive

Dividimos nuestro conjunto de train en 70 30

In [3]:
x_train, x_test, y_train, y_test = train_test_split(df_criticas.review_es,
                                                    df_criticas.sentimiento,
                                                    test_size=0.30,
                                                    random_state=25,
                                                    shuffle=True)

Antes de entrear un modelo para clasificacion necesitamos que este entrenamiento sea con valores numericos, y como nosotros tenemos palabras necesitamos convertirlas a vectores.

Utilizamos make_pipeline para aplicarle a nuestros modelos dos metodos distintos para vectorizar y luego el modelo de Naive Bayes Multidimensional. Una vez obtenidos los entrenamos

In [ ]:
model_NB = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_NB_2 = make_pipeline(CountVectorizer(), MultinomialNB())

model_NB.fit(x_train, y_train)
model_NB_2.fit(x_train, y_train)

categorias_predichas = model_NB.predict(x_test)
categorias_predichas_2 = model_NB_2.predict(x_test)


In [ ]:
print(accuracy_score(y_test, categorias_predichas))
print(accuracy_score(y_test, categorias_predichas_2))

0.8534666666666667
0.8376


Ya que el modelo 2 tuvo mejor accuracy es el que utilizamos para realizar la predicción de test

In [ ]:
criticas_test = df_criticas_test['review_es']
categorias_predichas_test = model_NB.predict(criticas_test)


Generamos el archivo de predicciones

In [ ]:
ids_test =  df_criticas_test.ID
filename = 'Naive_Bayes_Multidimensional'
prediccion = pd.DataFrame({"id": ids_test, "sentimiento": categorias_predichas_test})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)

# Random Forest

Optimizamos hiperparámetros

In [9]:
tree = RandomForestClassifier(random_state=1)
params_grid = {
    'bootstrap': [True, False],
    'ccp_alpha': [0.0, 0.01, 0.02],
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [10, 50, 100, 200],
}
rand = RandomizedSearchCV(tree, params_grid, cv=10, scoring='f1', n_iter=10, random_state=1)
rand.fit(x_train, y_train)
rand.cv_results_['mean_test_score']

array([0.66917622, 0.84101165, 0.76189623, 0.60415415, 0.79647694,
       0.68255784, 0.48244134, 0.67536519, 0.73080456, 0.69656625])

In [11]:
rand.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 30,
 'criterion': 'gini',
 'class_weight': None,
 'ccp_alpha': 0.0,
 'bootstrap': True}

Usamos el mejor estimador para predecir sobre el conjunto de training y obtenemos sus métricas

In [12]:
opt_tree = rand.best_estimator_
y_pred = opt_tree.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.79      0.83      7599
           1       0.80      0.87      0.84      7401

    accuracy                           0.83     15000
   macro avg       0.83      0.83      0.83     15000
weighted avg       0.83      0.83      0.83     15000



Finalmente predecimos sobre el conjunto de test y ajustamos la salida al formato aceptado por Kaggle

In [21]:
categorias_predichas_test = opt_tree.predict(X_test)

# Aplicar la función a cada elemento del array usando comprensión de listas
sentimientos = [convertir(valor) for valor in categorias_predichas_test]

In [22]:
filename = 'Random_Forest_opt'
prediccion = pd.DataFrame({"id": df_criticas_test.ID, "sentimiento": sentimientos})
prediccion.to_csv('/content/sample_data/prediccion_'+filename+'.csv', index = False)